In [ ]:
!pip install transformers
!pip install datasets
!pip install peft

In [1]:
# from google.colab import userdata
import os
from huggingface_hub import login
import pandas as pd
import datasets
from datasets import load_dataset,Dataset


In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-uncased")

/opt/conda/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [3]:
# from google.colab import files
import pandas as pd

ruta = "Data_Prueba/"
libros_lista = []
for i in os.listdir(ruta):
  libro = pd.read_csv(ruta + str(i),sep="\t",header=None,names=['texto'])
  # libro['libro'] = i
  libros_lista.append(libro)

In [4]:
libros = pd.concat(libros_lista, axis=0)

In [5]:
libros.shape

(3094, 1)

In [6]:
libros_completos = libros
libros = libros.iloc[:50]

In [7]:
libros = libros.set_index(['libro'], drop=True, append=False, inplace=False, verify_integrity=False)

KeyError: "None of ['libro'] are in the columns"

In [ ]:
libros

In [ ]:
dataset = Dataset.from_pandas(libros)

In [ ]:
print(dataset)

In [ ]:
from transformers import AutoConfig

# Download configuration from S3 and cache.
# config = AutoConfig.from_pretrained('bert-base-uncased')

# Download configuration from S3 (user-uploaded) and cache.
# config = AutoConfig.from_pretrained('dbmdz/bert-base-german-cased')

# If configuration file is in a directory (e.g., was saved using `save_pretrained('./test/saved_model/')`).
# config = AutoConfig.from_pretrained('./test/bert_saved_model/')

# Load a specific configuration file.
# config = AutoConfig.from_pretrained('./test/bert_saved_model/my_configuration.json')

# Change some config attributes when loading a pretrained config.
config = AutoConfig.from_pretrained('bert-base-uncased', output_attentions=True, foo=False)
# config.output_attentions
# config, unused_kwargs = AutoConfig.from_pretrained('bert-base-uncased', output_attentions=True, foo=False, return_unused_kwargs=True)
# config.output_attentions
# config.unused_kwargs

In [ ]:
# Integrar el modelo PEFT
# model = PeftModel(base_model, peft_config)
# Cargar los libros como dataset de texto
#dataset = load_dataset("text", data_files=data_files)

# Función para tokenizar el dataset
def tokenize_function(examples):
  return tokenizer(examples['texto'], truncation=True, padding="max_length", return_tensors='pt')

# # Tokenizar el dataset
tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Agregar las etiquetas al dataset tokenizado
def add_labels(examples):
  examples["labels"] = examples["input_ids"].copy()
  return examples

# Aplicar la función de etiquetas
tokenized_datasets = tokenized_datasets.map(add_labels, batched=True)

In [ ]:
from transformers import AutoModel, AutoTokenizer,BertForSequenceClassification
from transformers import BertModel

# Cargar el modelo Meta-Llama 3.1 8B en modo 8 bits
# model = model = AutoModel.from_pretrained('bert-base-uncased',num_labels=num_labels)
# from transformers import BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=512)

from peft import PeftModel, PeftConfig

# Configuración del modelo PEFT
peft_config = PeftConfig(
    base_model_name_or_path='bert-base-uncased'
    # adapter_name='my_adapter'
)

# Cargar el modelo base
base_model = BertForSequenceClassification.from_pretrained('bert-base-uncased')





In [ ]:
# Asegúrate de que tu dataset sea una instancia de Dataset
assert isinstance(tokenized_datasets, Dataset)

In [ ]:
tokenized_datasets[:25629]

In [ ]:
tokenized_datasets_train = tokenized_datasets[:25629]
# tokenized_datasets_val = tokenized_datasets.iloc[25629:,:]

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=1536,
    per_device_eval_batch_size=1536,
    gradient_accumulation_steps=8,
    num_train_epochs=3,
    learning_rate=5e-5,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    fp16=True,
    save_total_limit=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,  # Asegúrate de que esto sea un Dataset
    eval_dataset=tokenized_datasets,
)

trainer.train()


Esta parte del codigo es para agregarle un guardado automatico a los step

In [ ]:
# training_args = TrainingArguments(
#     output_dir="./results",
#     per_device_train_batch_size=1,
#     per_device_eval_batch_size=1,
#     gradient_accumulation_steps=8,
#     num_train_epochs=3,
#     learning_rate=5e-5,
#     weight_decay=0.01,
#     eval_strategy="epoch",
#     fp16=True,
#     save_total_limit=2,
#     save_steps=500,  # Guardar cada 500 pasos
#     save_strategy="steps"  # Usar la estrategia de guardar por pasos
# )

In [ ]:
trainer.save_model("./fine-tuned-llama-harry-potter")


Esta parte del codigo es para reanudarlo

In [ ]:
# # Cargar el modelo desde el último checkpoint
# model = AutoModelForCausalLM.from_pretrained("./checkpoint")

# # Continuar el entrenamiento
# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=tokenized_datasets["train"],
#     eval_dataset=tokenized_datasets["train"],
# )

# trainer.train(resume_from_checkpoint=True)

Probando modelo original

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# Cargar el modelo original (antes del fine-tuning)
model_original = AutoModelForCausalLM.from_pretrained("meta-llama/Meta-Llama-3.1-8B")
tokenizer_original = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3.1-8B")

# Asegurar que el tokenizador tiene el pad_token asignado
tokenizer_original.pad_token = tokenizer_original.eos_token

# Texto de prueba
input_text = "Harry Potter went to"

# Tokenizar el texto de entrada
inputs = tokenizer_original(input_text, return_tensors="pt")

# Generar texto con el modelo original
outputs_original = model_original.generate(**inputs, max_length=50)
generated_text_original = tokenizer_original.decode(outputs_original[0], skip_special_tokens=True)

print("Texto generado por el modelo original:")
print(generated_text_original)


descargar una carpeta entera

In [ ]:
# Comprimir la carpeta 'results' en un archivo zip
!zip -r results.zip /content/results


In [ ]:
from google.colab import files
files.download('results.zip')


In [ ]:
# Comprimir la carpeta 'results' en un archivo zip
!zip -r fine-tuned-llama-harry-potter.zip /content/fine-tuned-llama-harry-potter

from google.colab import files
files.download('fine-tuned-llama-harry-potter.zip')

In [ ]:
!ls /root/.cache/huggingface/transformers


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

# Cargar el modelo fine-tuned usando LoRA
model_finetuned = AutoModelForCausalLM.from_pretrained("./fine-tuned-llama-harry-potter")  # Ruta donde guardaste el modelo entrenado

# Cargar el tokenizador
tokenizer = AutoTokenizer.from_pretrained("./fine-tuned-llama-harry-potter")  # Ruta del tokenizador guardado junto al modelo
tokenizer.pad_token = tokenizer.eos_token

# Texto de entrada para probar
input_text = "Harry Potter went to"

# Tokenizar el texto de entrada
inputs = tokenizer(input_text, return_tensors="pt")

# Generar texto con el modelo fine-tuned
outputs = model_finetuned.generate(**inputs, max_length=5) # Era 50 y se queda en 25%

# Decodificar los tokens generados a texto legible
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

print("Texto generado por el modelo entrenado (fine-tuned):")
print(generated_text)


In [ ]:
!pwd